In [2]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers 
import time
import dataset
import glob

In [3]:
discriminator= keras.Sequential(
    [
        keras.Input(shape=(64, 64,3)),
        layers.Conv2D(32, (3, 3), strides=1, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.AveragePooling2D(strides=2),
        layers.Conv2D(64, (3, 3), strides=1, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.AveragePooling2D(strides=2),
        layers.Conv2D(64, (3, 3), strides=1, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.AveragePooling2D(strides=2),
        layers.Conv2D(128, (3, 3), strides=1, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.AveragePooling2D(strides=2),
        layers.Conv2D(128, 2, strides=1, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.AveragePooling2D(strides=2),
        layers.Flatten(),
        layers.Dense(1),
    ],
    name="discriminator",
)

In [4]:
latent_dim = 128

In [5]:


latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(2 * 2* 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(4 * 4 * 256),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((4, 4, 256)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,2,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,2,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (7, 7), padding="same", activation="sigmoid"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,2,1,padding="same"),
        # layers.LeakyReLU(alpha=0.2),
        # layers.Conv2D(128,2,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        # layers.LeakyReLU(alpha=0.2),
        # layers.Conv2D(128,2,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,2,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, 8, padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [6]:
discriminator=keras.models.load_model("./face_dis")
generator=keras.models.load_model("./face_gen/")


In [7]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.RMSprop(learning_rate=0.0008)
g_optimizer = keras.optimizers.RMSprop(learning_rate=0.00008)

# Instantiate a loss function.
# loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    for i in range(3):
        
        random_vector=tf.random.normal(shape=[real_images.shape[0],latent_dim])
        with tf.GradientTape() as tape1:
            with tf.GradientTape() as tape2: 
                generate_image_1=generator(random_vector)
                alph=tf.random.normal(shape=[real_images.shape[0],1,1,1])
                generate_image_2=alph*real_images+(1-alph)*generate_image_1
                out_real=discriminator(real_images)
                out_gen_1=discriminator(generate_image_1)
                out_gen_2=discriminator(generate_image_2)
            grade_1=tape2.gradient(out_gen_2,generate_image_2)
            grade_l2=gradien_panalty_loss(grade_1)
            loss_dis=dis_loss(out_real,out_gen_1,grade_l2,20)
        grade_2=tape1.gradient(loss_dis,discriminator.trainable_variables)
        d_optimizer.apply_gradients(zip(grade_2,discriminator.trainable_variables))
    for i in range(1):
        random_vector=tf.random.normal(shape=[real_images.shape[0],latent_dim])
        with tf.GradientTape() as tape:
            generate_image= generator(random_vector)
            out=discriminator(generate_image)
            loss_gen=gen_loss(out)
        grade=tape.gradient(loss_gen,generator.trainable_variables)
        g_optimizer.apply_gradients(zip(grade,generator.trainable_variables))
    return loss_dis,loss_gen, generate_image

In [8]:
@tf.function
def dis_loss(real,fake,grade,lamda):
    loss=tf.reduce_mean(fake)-tf.reduce_mean(real)+lamda*grade
    return loss
@ tf.function
def gen_loss(fake):
    return -tf.reduce_mean(fake) 
@ tf.function
def gradien_panalty_loss(grade):
    grad_sqr=tf.square(grade)
    grad_sqr_sum=tf.reduce_sum(grad_sqr,axis=np.arange(1,len(grad_sqr.shape)))
    grad_l2_norm=tf.sqrt(grad_sqr_sum)
    grad_penalty=tf.square(1-grad_l2_norm)
    return tf.reduce_mean(grad_penalty)


In [9]:
import os

# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
# (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# all_digits = np.concatenate([x_train, x_test])
# all_label=keras.utils.to_categorical( np.concatenate([y_train,y_test]))
# all_digits = all_digits.astype("float32") / 255.0
# dataset = tf.data.Dataset.from_tensor_slices((all_digits,all_label))
img_path = glob.glob(r'F:/GAN/faces/*.jpg')
dataset1, img_shape, _ = dataset.make_anime_dataset(img_path, batch_size,64)



In [7]:

epochs = 2000       # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./"

for epoch in range(epochs):
    print("\nStart epoch", epoch)
    dataset1=dataset1.shuffle(1024)
    for step, real_images in enumerate(dataset1):
        # break
        # Train the discriminator & generator on one batch of real images.
        # rand_label1=keras.utils.to_categorical(np.random.randint(10,size=(batch_size,1)))
        # tf.random.set_seed(int(time.time()))
        d_loss, g_loss, generated_images= train_step(real_images)
        # print(real_images.shape)
        # Logging.
        if step % 200 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img_wgan_" + str(step) + ".png"))
        #     # print(np.argmax(rand_label1,axis=-1)[0])
        # # To limit execution time we stop after 10 steps.
        # # Remove the lines below to actually train the model!
        # # if step > 10:
        # #     break
    if epoch%50==0:
       discriminator.save("./face_dis")
       generator.save("./face_gen")
        



Start epoch 0
discriminator loss at step 0: -8.22
adversarial loss at step 0: 27.50
discriminator loss at step 200: -8.61
adversarial loss at step 200: 50.62
discriminator loss at step 400: -9.01
adversarial loss at step 400: 34.47
discriminator loss at step 600: -8.59
adversarial loss at step 600: 31.77
discriminator loss at step 800: -9.50
adversarial loss at step 800: 48.99
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./face_dis\assets
INFO:tensorflow:Assets written to: ./face_gen\assets

Start epoch 1
discriminator loss at step 0: -9.22
adversarial loss at step 0: 49.84
discriminator loss at step 200: -7.98
adversarial loss at step 200: 46.42
discriminator loss at step 400: -8.74
adversarial loss at step 400: 44.96
discriminator loss at step 600: -9.68
adversarial loss at step 600: 33.60
discriminator loss at step 800: -9.17
adversarial loss at step 800: 54.10

Start epoch 2
discriminator loss at step 0: -9.23


KeyboardInterrupt: 

In [ ]:

# plt.figure()
# for i in range(1,17):
#     plt.subplot(4,4,i)
#     plt.imshow(next(iter(dataset1))[i,...])
a=next(iter(dataset1))[i,...]


In [ ]:
a=tf.Variable([[1,2],[2,3]])
b=tf.Variable([[2],[3]])
a*b

In [ ]:

with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2: 
        random_vector=tf.random.normal(shape=[real_images.shape[0],latent_dim])
        generate_image_1=generator(random_vector)
        alph=tf.random.normal(shape=[real_images.shape[0],1,1,1])
        generate_image_2=alph*real_images+(1-alph)*generate_image_1
        out_real=discriminator(real_images)
        out_gen_1=discriminator(generate_image_1)
        out_gen_2=discriminator(generate_image_2)
    grade_1=tape2.gradient(out_gen_2,generate_image_2)
    grade_l2=gradien_panalty_loss(grade_1)
    loss_dis=dis_loss(out_real,out_gen_1,grade_l2,10)
grade_2=tape1.gradient(loss_dis,discriminator.trainable_variables)

In [1]:
import os 
os.mkdir("./face_dis2")
os.mkdir("./face_gen2")